In [4]:
import utility as ut

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


import pandas as pd

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D

In [5]:
df_full = pd.read_csv('df_full.csv')

In [6]:
df_model = df_full.drop(['Unnamed: 0', 'student_id'], axis=1)

# Identify categorical and numeric columns
categorical_cols = df_model.select_dtypes(include=['object']).columns.drop('status')
numeric_cols = df_model.select_dtypes(include=['int64', 'float64']).columns

# Define a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Separate the target variable
X = df_model.drop('status', axis=1)
y = df_model['status']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the preprocessing pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
X_train_prepared = pipeline.fit_transform(X_train)
X_test_prepared = pipeline.transform(X_test)

# Check the shapes of the processed data
X_train_prepared.shape, X_test_prepared.shape

((13134, 17), (3284, 17))

In [7]:
X_train_reshaped = X_train_prepared.reshape(X_train_prepared.shape[0], 1, X_train_prepared.shape[1])
X_test_reshaped = X_test_prepared.reshape(X_test_prepared.shape[0], 1, X_test_prepared.shape[1])


In [8]:

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)


In [9]:


model = Sequential([
    Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, X_train_reshaped.shape[2])),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(y_train_categorical.shape[1], activation='softmax')
])


/opt/anaconda3/envs/dataFest/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_reshaped, y_train_categorical, epochs=10, validation_data=(X_test_reshaped, y_test_categorical))

loss, accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print("Test Accuracy:", accuracy)


Epoch 1/10
